# Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import pyperclip
import time
import random
import pandas as pd
import re

# Crawling

In [51]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

# 로그인
driver.get("https://everytime.kr/login")

# 사용자 자격 증명
nid = '' # 내 ID
pyperclip.copy(nid)
driver.find_element(By.NAME, "id").send_keys(Keys.CONTROL + 'v')
npw = '' # 내 비밀번호
pyperclip.copy(npw)
secure = 'blank'
driver.find_element(By.NAME, "password").send_keys(Keys.CONTROL + 'v')
pyperclip.copy(secure)

# 로그인 양식 제출
driver.find_element(By.XPATH, '/html/body/div/div/form/input').click()

# 로그인 완료까지 대기
wait.until(EC.url_changes("https://everytime.kr/login"))

# 크롤링 루프
titles, comments, votes, comment_nums, scraps = [], [], [], [], []

'''자유게시판(389113): 14,160 x 20 = 약 283,200
비밀게시판(258806): 2,095 x 20 = 약 41,900
졸업생게시판(389397): 118 x 20 = 약 2,360
새내기게시판(389202): 2,228 x 20 = 약 44,560
시사이슈(482609): 169 x 20 = 약 3,380
장터게시판(389346): 335 x 20 = 약 6,700
정보게시판(258808): 7 x 20 = 약 140
홍보게시판(367467): 106 x 20 = 약 2,120
취업진로(389293): 28 x 20 = 약 560
제목, 본문 내용, 투표수, 댓글수, 스크랩수'''

driver.get(f"https://everytime.kr/389113/p/1")
time.sleep(3)

iterable = range(1, 2000)
pbar = tqdm(iterable,
            total=len(iterable),
            desc='page',
            ncols=100,
            ascii=' =',
            leave=True)

for cnt in pbar: 
    # everytime 게시판 링크
    pbar.set_description(f'Current Page "{cnt}"')
    driver.get(f"https://everytime.kr/389113/p/{cnt}")

    # 페이지가 로드될 때까지 대기
    time.sleep(random.randrange(1, 7))

    # 링크 가져오기
    posts = driver.find_elements(By.CSS_SELECTOR, 'div > article > a.article')
    links = [post.get_attribute('href') for post in posts]

    # 상세 글 가져오기
    for link in tqdm(links, desc='link', position=1, leave=False):
        driver.get(link)
        # 게시판 원문 가져오기
        try:
            # 페이지가 로드될 때까지 대기
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.large')))
            titles.append(driver.find_element(By.CSS_SELECTOR, '#container > div.wrap.articles > article > a > h2.large').text)
            comments.append(driver.find_element(By.CSS_SELECTOR, "#container > div.wrap.articles > article > a > p.large").text)
            votes.append(driver.find_element(By.CSS_SELECTOR, '#container > div.wrap.articles > article > a > ul.status.left > li.vote').text)
            comment_nums.append(driver.find_element(By.CSS_SELECTOR, '#container > div.wrap.articles > article > a > ul.status.left > li.comment').text)
            scraps.append(driver.find_element(By.CSS_SELECTOR, '#container > div.wrap.articles > article > a > ul.status.left > li.scrap').text)
        except:
            continue
        
# 브라우저 닫기
driver.quit()

results = pd.DataFrame({'title': titles, 'main': comments, 'votes': votes, 'comment_nums': comment_nums, 'scraps': scraps})

results.to_csv('results_free.csv')

Current Page "1999": 100%|====================================| 1000/1000 [6:22:41<00:00, 22.96s/it]


# Data Preprocessing

In [ ]:
def clean_str(text):
    # 여러 패턴을 하나로 통합
    patterns = [
        '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)',  # E-mail 제거
        '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+',  # URL 제거
        '([ㄱ-ㅎㅏ-ㅣ]+)',  # 한글 자음, 모음 제거
        '<[^>]*>',  # HTML 태그 제거
        '[^\w\s\n]',  # 특수 기호 제거
        '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '',  # 추가 특수 기호 제거
        '\n', '.'  # 줄 바꿈을 마침표로 변경
    ]
    combined_pattern = '|'.join(patterns)
    text = re.sub(pattern=combined_pattern, repl='', string=text)
    text = re.sub('[一-龯a-zA-Z]', '', string=text)  # 한자 및 영문 제거
    return text

def prepro_df(df):
    # 'title'이 NaN인 경우에 대비하여 fillna 사용
    df['content'] = df['title'].fillna('') + ' ' + df['main']
    df = df.drop(columns=['Unnamed: 0', 'title', 'main'])
    df = df.sample(frac=1).reset_index(drop=True)
    df['content'] = df['content'].apply(clean_str)
    return df